# Welcome to DFL-Colab!

This is an adapted version of the DFL for Google Colab.

Version 2.5

# Overview
*   Extractor works in full functionality.
*   Training can work with preview or without it.
*   Converter works in full functionality.
*   You can import/export workspace with your Google Drive.
*   Import/export and another manipulations with workspace you can do in "Manage workspace" block
*   Google Colab machine active for 12 hours. DFL-Colab makes a backup of your workspace in training mode, after 11 hours from the start of the session.
*   Google does not like long-term heavy calculations. Therefore, for training more than two sessions in a row, use two Google accounts. It is recommended to split your training over 2 accounts, but you can use one Google Drive account to store your workspace.



# Clone Github repository and install requirements

* Clone Github repository or pull updates
* Requirements install is automatically

In [0]:
#@title Clone or pull DeepFaceLab from Github

Mode = "clone" #@param ["clone", "pull"]

from pathlib import Path
if (Mode == "clone"):
  !git clone https://github.com/iperov/DeepFaceLab.git
else:
  %cd /content/DeepFaceLab
  !git pull

!pip install -r /content/DeepFaceLab/requirements-colab.txt
!pip install --upgrade scikit-image
!pip install watchdog

if not Path("/content/workspace").exists():
  !wget -q --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=1hTH2h6l_4kKrczA8EkN6GyuXx4lzmCnK' -O pretrain_CelebA.zip
  !mkdir /content/pretrain
  !unzip -q /content/pretrain_CelebA.zip -d /content/pretrain/
  !rm /content/pretrain_CelebA.zip

print("Done!")

#Manage workspace



*   You can import/export workspace or individual data, like model files with Google Drive
*   Also, you can use HFS (HTTP Fileserver) for directly import/export you workspace from your computer
*   You can clear all workspace or delete part of it



In [0]:
#@title Import from Drive

Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "workspace.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
  
print("Done!")



In [0]:
#@title Export to Drive { form-width: "30%" }
Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "merged", "models"]
Archive_name = "workspace.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  zip_cmd="-r -q "+Archive_name+" "
  
  %cd $path
  zip_cmd+=mode
  !zip $zip_cmd
  copy_cmd = " "+Archive_name+"  /content/drive/My\ Drive/"
  !cp $copy_cmd
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "merged":
  zip_and_copy("/content/workspace/data_dst", "merged")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
  
print("Done!")


In [0]:
#@title Import from URL{ form-width: "30%", display-mode: "form" }
URL = "http://" #@param {type:"string"}
Mode = "unzip to content/workspace" #@param ["unzip to content", "unzip to content/workspace", "unzip to content/workspace/data_src", "unzip to content/workspace/data_src/aligned", "unzip to content/workspace/data_dst", "unzip to content/workspace/data_dst/aligned", "unzip to content/workspace/model", "download to content/workspace"]

import urllib
from pathlib import Path

def unzip(zip_path, dest_path):

    
  unzip_cmd = " unzip -q " + zip_path + " -d "+dest_path
  !$unzip_cmd  
  rm_cmd = "rm "+dest_path + url_path.name
  !$rm_cmd
  print("Unziped!")
  

if Mode == "unzip to content":
  dest_path = "/content/"
elif Mode == "unzip to content/workspace":
  dest_path = "/content/workspace/"
elif Mode == "unzip to content/workspace/data_src":
  dest_path = "/content/workspace/data_src/"
elif Mode == "unzip to content/workspace/data_src/aligned":
  dest_path = "/content/workspace/data_src/aligned/"
elif Mode == "unzip to content/workspace/data_dst":
  dest_path = "/content/workspace/data_dst/"
elif Mode == "unzip to content/workspace/data_dst/aligned":
  dest_path = "/content/workspace/data_dst/aligned/"
elif Mode == "unzip to content/workspace/model":
  dest_path = "/content/workspace/model/"
elif Mode == "download to content/workspace":
  dest_path = "/content/workspace/"

if not Path("/content/workspace").exists():
  cmd = "mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"
  !$cmd

url_path = Path(URL)
urllib.request.urlretrieve ( URL, dest_path + url_path.name )

if (url_path.suffix == ".zip") and (Mode!="download to content/workspace"):
  unzip(dest_path + url_path.name, dest_path)

  
print("Done!")

In [0]:
#@title Export to URL
URL = "http://" #@param {type:"string"}
Mode = "upload workspace" #@param ["upload workspace", "upload data_src", "upload data_dst", "upload data_src aligned", "upload data_dst aligned", "upload merged", "upload model"]

cmd_zip = "zip -r -q "

def run_cmd(zip_path, curl_url):
  cmd_zip = "zip -r -q "+zip_path
  cmd_curl = "curl --silent -F "+curl_url+" -D out.txt > /dev/null"
  !$cmd_zip
  !$cmd_curl


if Mode == "upload workspace":
  %cd "/content"
  run_cmd("workspace.zip workspace/","'data=@/content/workspace.zip' "+URL)
elif Mode == "upload data_src":
  %cd "/content/workspace"
  run_cmd("data_src.zip data_src/", "'data=@/content/workspace/data_src.zip' "+URL)
elif Mode == "upload data_dst":
  %cd "/content/workspace"
  run_cmd("data_dst.zip data_dst/", "'data=@/content/workspace/data_dst.zip' "+URL)
elif Mode == "upload data_src aligned":
  %cd "/content/workspace"
  run_cmd("data_src_aligned.zip data_src/aligned", "'data=@/content/workspace/data_src_aligned.zip' "+URL )
elif Mode == "upload data_dst aligned":
  %cd "/content/workspace"
  run_cmd("data_dst_aligned.zip data_dst/aligned/", "'data=@/content/workspace/data_dst_aligned.zip' "+URL)
elif Mode == "upload merged":
  %cd "/content/workspace/data_dst"
  run_cmd("merged.zip merged/","'data=@/content/workspace/data_dst/merged.zip' "+URL )
elif Mode == "upload model":
  %cd "/content/workspace"
  run_cmd("model.zip model/", "'data=@/content/workspace/model.zip' "+URL)
  
  
!rm *.zip

%cd "/content"
print("Done!")

In [0]:
#@title Delete and recreate
Mode = "Delete data_src aligned" #@param ["Delete and recreate workspace", "Delete models", "Delete data_src", "Delete data_src aligned", "Delete data_src video", "Delete data_dst", "Delete data_dst aligned", "Delete merged frames"]

%cd "/content" 

if Mode == "Delete and recreate workspace":
  cmd = "rm -r /content/workspace ; mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"  
elif Mode == "Delete models":
  cmd = "rm -r /content/workspace/model/*"
elif Mode == "Delete data_src":
  cmd = "rm /content/workspace/data_src/*.png || rm -r /content/workspace/data_src/*.jpg"
elif Mode == "Delete data_src aligned":
  cmd = "rm -r /content/workspace/data_src/aligned/*"
elif Mode == "Delete data_src video":
  cmd = "rm -r /content/workspace/data_src.*"
elif Mode == "Delete data_dst":
  cmd = "rm /content/workspace/data_dst/*.png || rm /content/workspace/data_dst/*.jpg"
elif Mode == "Delete data_dst aligned":
  cmd = "rm -r /content/workspace/data_dst/aligned/*"
elif Mode == "Delete merged frames":
  cmd = "rm -r /content/workspace/data_dst/merged"
  
!$cmd
print("Done!")

# Extract and sorting
* Extract frames for SRC or DST video.
* Denoise SRC or DST video. "Factor" param set intesity of denoising
*   Detect and align faces with one of detectors. (S3FD - recommended). If you need, you can get frames with debug landmarks.
*   Export workspace to Google Drive after extract and sort it manually (Last block of Notebook)


In [0]:
#@title Extract frames
Video = "data_src" #@param ["data_src", "data_dst"]

%cd "/content"

cmd = "DeepFaceLab/main.py videoed extract-video"

if Video == "data_dst":
  cmd+= " --input-file workspace/data_dst.* --output-dir workspace/data_dst/"
else:
  cmd+= " --input-file workspace/data_src.* --output-dir workspace/data_src/"
  
!python $cmd

In [0]:
#@title Denoise frames
Data = "data_src" #@param ["data_src", "data_dst"]
Factor = 1 #@param {type:"slider", min:1, max:20, step:1}

cmd = "DeepFaceLab/main.py videoed denoise-image-sequence --input-dir workspace/"+Data+" --factor "+str(Factor)

%cd "/content"
!python $cmd

In [0]:
#@title Detect faces
Data = "data_src" #@param ["data_src", "data_dst"]
Detector = "S3FD" #@param ["S3FD", "MT"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
if Detector == "S3FD":
  detect_type = "s3fd"
elif Detector == "MT":
  detect_type = "mt"

folder = "workspace/"+Data
folder_align = folder+"/aligned"
debug_folder = folder_align+"/debug"

cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_align

if Debug:
  cmd+= " --debug-dir "+debug_folder

cmd+=" --detector "+detect_type
  
%cd "/content"
!python $cmd

In [0]:
#@title Sort aligned
Data = "data_src" #@param ["data_src", "data_dst"]
sort_type = "hist" #@param ["hist", "hist-dissim", "face-yaw", "face-pitch", "blur", "final"]

cmd = "DeepFaceLab/main.py sort --input-dir workspace/"+Data+"/aligned --by "+sort_type

%cd "/content"
!python $cmd

# Train model

*   Choose your model type, but SAE is recommend for everyone
*   Set model options on output field
*   You can get preview in output if choose option "Preview". Or you can see preview manually, if go to model folder in filemanager and double click on preview.jpg file
* Your workspace will be archived and upload to mounted Drive after 11 hours from start session
* Also, you can export your workspace in "Manage workspace" block

In [0]:
#@title Training
Model = "SAE" #@param ["SAE", "H128", "LIAEF128", "DF", "DEV_FANSEG", "RecycleGAN"]
Preview = False #@param {type:"boolean"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

import psutil, os, time
import multiprocessing as mp
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from IPython.display import Image
from IPython.display import clear_output

################################################################################
# Preview Image Watcher
################################################################################
class PreviewImageWatcher:
    DIRECTORY_TO_WATCH = "/content/workspace/model/"

    def __init__(self):
        self.observer = Observer()

    def run(self):
        event_handler = Handler()
        self.observer.schedule(event_handler, self.DIRECTORY_TO_WATCH, recursive=True)
        self.observer.start()
        try:
            while True:
                time.sleep(5)
        except:
            self.observer.stop()
            print("PreviewImageWatcher stopped!")

        self.observer.join()


class Handler(FileSystemEventHandler):
    @staticmethod
    def on_any_event(event):
        if event.is_directory:
            return None
        elif event.event_type == 'modified':
            if (event.src_path.endswith(".png") or event.src_path.endswith(".jpg")):
                if (not 'masked' in event.src_path):
                    clear_output()
                    display(Image(event.src_path))


def watch():
    w = PreviewImageWatcher()
    print("Starting Preview Image Watcher")
    w.run()


################################################################################
# Run preview image watcher as async process
################################################################################
if Preview:
  processpool = mp.Pool(processes=1)
  processpool.apply_async(watch)

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

if (round(39600-uptime) > 0):
  backup_time = str(round(39600-uptime))
  print("Time to backup: "+str(round((39600-uptime)/3600))+" hours")
  cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain/aligned --model-dir workspace/model --model "+Model
  backup_cmd = " --execute-program "+backup_time+" \"import os; os.system('zip -r -q workspace.zip workspace'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print(' Backuped!') \"" 
else:
  cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model
  backup_cmd = ""

if (backup_cmd != ""):
  train_cmd = (cmd+backup_cmd)
else:
  train_cmd = (cmd)

%cd "/content"
!python $train_cmd

# Convert frames

In [0]:
#@title Convert
Model = "SAE" #@param ["SAE", "H128", "LIAEF128", "DF", "RecycleGAN"]

cmd = "DeepFaceLab/main.py convert --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [0]:
#@title Get result video and copy to Drive 

!python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4
!cp /content/workspace/result.mp4 /content/drive/My\ Drive/